In [222]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import missingno as miss
from scipy import stats
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.model_selection import cross_val_score,cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.exceptions import FitFailedWarning
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

train=pd.read_csv("/Users/saranlalmokandasan/Downloads/train_ctrUa4K.csv")
test=pd.read_csv("/Users/saranlalmokandasan/Downloads/test_lAUu6dG.csv")
testcol=test["Loan_ID"]
train=train[train.LoanAmount<600]
train.Self_Employed.fillna("No",inplace=True)
train.Credit_History.fillna("1.0",inplace=True)
train.loc[train.Credit_History!=0,'Credit_History']=1
train.Loan_Amount_Term.fillna("360",inplace=True)
train.Dependents.fillna("0",inplace=True)
train.Dependents.replace("3+",3,inplace=True)
train.Gender.fillna('Male',inplace=True)
train.Married.fillna('Y',inplace=True)
train.LoanAmount.fillna(np.median(train.LoanAmount),inplace=True)
train.insert(10,"Loan_Risk","other")
train.loc[(train["Credit_History"]==1)&(train["Dependents"]=="0")&(train["Self_Employed"]=="No")&(train["Property_Area"]=="Semiurban"),"Loan_Risk"]=0
train.loc[train["Loan_Risk"]=="other","Loan_Risk"]=1

train.drop("Loan_ID",axis=1,inplace=True)
train.insert(2,"Debt_Income",(train.LoanAmount.astype(float)/(train.ApplicantIncome+train.CoapplicantIncome)))


'''
train.insert(5,"Total_Income",train["ApplicantIncome"]+train["CoapplicantIncome"])
train.insert(4,"Status","other")
train.loc[train["Total_Income"]>=20000,"Status"]="Very High"

train.loc[(train["Total_Income"]>=10000)&(train["Total_Income"]<20000),"Status"]="High"
train.loc[(train["Total_Income"]>=7000)&(train["Total_Income"]<10000),"Status"]="Medium"
train.loc[(train["Total_Income"]>=5000)&(train["Total_Income"]<7000),"Status"]="Low"
train.loc[(train["Total_Income"]>=1400)&(train["Total_Income"]<5000),"Status"]="Very Low"
train.drop(columns=["ApplicantIncome","CoapplicantIncome"],axis=1,inplace=True)
'''



categorical_feature_mask = train.dtypes==object
categorical_cols = train.columns[categorical_feature_mask].tolist()
le = preprocessing.LabelEncoder()
train[categorical_cols] = train[categorical_cols].apply(lambda col: le.fit_transform(col.astype(str)))
test.Self_Employed.fillna("No",inplace=True)
test.Gender.fillna('Male',inplace=True)
test.Married.fillna('Y',inplace=True)
test.Dependents.fillna("0",inplace=True)
test.Loan_Amount_Term.fillna("360",inplace=True)
test.Credit_History.fillna(1.0,inplace=True)
train.loc[train.Credit_History!=0,'Credit_History']=1
test.LoanAmount.fillna(np.mean(test.LoanAmount),inplace=True)

test.insert(10,"Loan_Risk","other")
test.loc[(test["Credit_History"]==1)&(test["Dependents"]=="0")&(test["Self_Employed"]=="No")&(train["Property_Area"]=="Semiurban"),"Loan_Risk"]=0
test.loc[test["Loan_Risk"]=="other","Loan_Risk"]=1


        


test.drop("Loan_ID",axis=1,inplace=True)
test.insert(2,"Debt_Income",(test.LoanAmount/(test.ApplicantIncome+(test.CoapplicantIncome))))

'''
test.insert(5,"Total_Income",test["ApplicantIncome"]+test["CoapplicantIncome"])
test.insert(4,"Status","other")
test.loc[test["Total_Income"]>=20000,"Status"]="Very High"
test.loc[(test["Total_Income"]>=10000)&(test["Total_Income"]<20000),"Status"]="High"
test.loc[(test["Total_Income"]>=7000)&(test["Total_Income"]<10000),"Status"]="Medium"
test.loc[(test["Total_Income"]>=5000)&(test["Total_Income"]<7000),"Status"]="Low"
test.loc[(test["Total_Income"]>=1400)&(test["Total_Income"]<5000),"Status"]="Very Low"
test.drop(columns=["ApplicantIncome","CoapplicantIncome"],axis=1,inplace=True)
'''


categorical_feature_masks=test.dtypes==object
categorical_cols=test.columns[categorical_feature_masks].tolist()
test[categorical_cols]=test[categorical_cols].apply(lambda col:le.fit_transform(col.astype(str)))
X=train.iloc[:,0:13]
y=train.Loan_Status
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
accuracy = cross_val_score(classifier, X, y, scoring='accuracy', cv = 6)
print("The Accuracy of Random Forrest is : {}".format(accuracy.mean()))
lr=LogisticRegression(solver='liblinear',max_iter = 1000)
accuracy=cross_val_score(lr,X,y,scoring='accuracy',cv=6)
print("The Accuracy of Logistic Regression is : {}".format(accuracy.mean()))
nm=GaussianNB()
accuracy=cross_val_score(nm,X,y,scoring='accuracy',cv=6)
print("The Accuracy of Naive Bayes is : {}".format(accuracy.mean()))
classi=SGDClassifier()
accuracy=cross_val_score(classi,X,y,scoring='accuracy',cv=6)
print("The Accuracy of SGDClassifier is : {}".format(accuracy.mean()))
neigh = KNeighborsClassifier(n_neighbors=3)
accuracy=cross_val_score(neigh,X,y,scoring='accuracy',cv=6)
print("The Accuracy of knn is : {}".format(accuracy.mean()))
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,stratify=y)
model1 = LogisticRegression(random_state=1,solver='liblinear',max_iter = 1000)
model2 = RandomForestClassifier(random_state=1)
model3= nm=GaussianNB()
model = VotingClassifier(estimators=[('lr', model1), ('dt', model2),('nei',model3)], voting='hard')
model.fit(X,y)
model.predict(test)
print("The Accuracy score of Voting Classifier is {}".format(model.score(x_test,y_test)))
estimators = [('rf', LogisticRegression(solver='lbfgs',max_iter = 1000)),
('svr', make_pipeline(StandardScaler(),LinearSVC(random_state=42,dual=False)))]
clf = StackingClassifier(estimators=estimators, final_estimator=RandomForestClassifier(n_estimators=10, random_state=42))
clf.fit(x_train, y_train)
print("The Accuracy score of Stacking Classifier is {}".format(clf.score(x_test,y_test)))

rfc=RandomForestClassifier(criterion= 'gini', max_depth= 5, max_features= 'auto', n_estimators= 200,random_state=42)
rfc.fit(X,y)
rfc.predict(test)
lr=LogisticRegression(solver='liblinear',max_iter = 1000)
lr.fit(X,y)
lr.predict(test)
submissions=pd.DataFrame({"Loan_ID":testcol, "Loan_Status":final})
submissions.Loan_Status.replace({1:"Y",0:"N"},inplace=True)
submissions.to_csv('submissions.csv',index=False)

# Create first pipeline for base without reducing features.


# pipe = Pipeline([('classifier', RandomForestClassifier())])

# Create param grid.

#param_grid = [
 #   {
  #   'penalty' : ['l1', 'l2'],
   # 'C' : np.logspace(-4, 4, 20),
    #'solver' : ['liblinear','newton-cg','lbfgs','sag','saga'  ]},
   
#]

# Create grid search object

#clfs = GridSearchCV(LogisticRegression(max_iter = 2000), param_grid = param_grid, cv = 5, verbose=True, n_jobs=-1)

# Fit on data
clfs=LogisticRegression(max_iter=2000,solver='liblinear',penalty='l2',C= 0.615848211066026)

clfs= clfs.fit(x_train, y_train)

final=clfs.predict(test)


submissions=pd.DataFrame({"Loan_ID":testcol, "Loan_Status":final})
submissions.Loan_Status.replace({1:"Y",0:"N"},inplace=True)
submissions.to_csv('submissions.csv',index=False)
print("Hi")



The Accuracy of Random Forrest is : 0.8078231292517005
The Accuracy of Logistic Regression is : 0.814625850340136
The Accuracy of Naive Bayes is : 0.7976190476190476
The Accuracy of SGDClassifier is : 0.5901360544217688
The Accuracy of knn is : 0.6224489795918366
The Accuracy score of Voting Classifier is 0.847457627118644
The Accuracy score of Stacking Classifier is 0.7966101694915254
Hi
